## Vanilla GAN

> Reference: https://github.com/wiseodd/generative-models/tree/master/GAN/vanilla_gan

> Comment added by: YQ

#### 1. import modules

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

#### 2. define function: Xavier Initialization for weights (parameters)

In [2]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

#### 3. define the placeholders (inputs) and variables (parameters) for Generative Network and Discriminative Network   

In [3]:
X = tf.placeholder(tf.float32, shape=[None, 784])

D_W1 = tf.Variable(xavier_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


Z = tf.placeholder(tf.float32, shape=[None, 100])

G_W1 = tf.Variable(xavier_init([100, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(xavier_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

#### 4. Define the Networks

In [4]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob


def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit


def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig


#### 5. Call the functions and assign the outputs to 4 variables, the variables will be used for loss calculation

#### Then, the loss functions will be used for loss calculation, which will be used for optimizers and their training

In [5]:
G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

# D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
# G_loss = -tf.reduce_mean(tf.log(D_fake))

# Alternative losses:
# -------------------
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)


#### 6. define the minibatch size as well as the Z_dim ( whats this? )

#### import mnist from Keras

#### initialize tensorflow session

#### create the folder for generated images

In [6]:
mb_size = 128
Z_dim = 100

mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0


Extracting ../../MNIST_data\train-images-idx3-ubyte.gz
Extracting ../../MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data\t10k-labels-idx1-ubyte.gz


#### 7. trian the model according the the epoch number defined

#### save the output image from generator after each 1000 epoches

#### print loss for each 1000 epoches

In [7]:
for it in range(100000):
    if it % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()


Iter: 0
D loss: 1.414
G_loss: 2.62

Iter: 1000
D loss: 0.01008
G_loss: 8.75

Iter: 2000
D loss: 0.05239
G_loss: 6.03

Iter: 3000
D loss: 0.07124
G_loss: 4.747

Iter: 4000
D loss: 0.1842
G_loss: 6.069

Iter: 5000
D loss: 0.455
G_loss: 4.588

Iter: 6000
D loss: 0.5263
G_loss: 4.118

Iter: 7000
D loss: 0.4501
G_loss: 3.674

Iter: 8000
D loss: 0.5249
G_loss: 2.968

Iter: 9000
D loss: 0.5193
G_loss: 2.991

Iter: 10000
D loss: 0.4717
G_loss: 2.722

Iter: 11000
D loss: 0.5366
G_loss: 3.063

Iter: 12000
D loss: 0.5429
G_loss: 2.579

Iter: 13000
D loss: 0.7747
G_loss: 2.133

Iter: 14000
D loss: 0.6218
G_loss: 2.581

Iter: 15000
D loss: 0.7816
G_loss: 2.118

Iter: 16000
D loss: 0.711
G_loss: 2.794

Iter: 17000
D loss: 0.8173
G_loss: 2.393

Iter: 18000
D loss: 0.6144
G_loss: 2.582

Iter: 19000
D loss: 0.4749
G_loss: 2.774

Iter: 20000
D loss: 0.8724
G_loss: 2.135

Iter: 21000
D loss: 0.8655
G_loss: 2.491

Iter: 22000
D loss: 0.6507
G_loss: 2.419

Iter: 23000
D loss: 0.8215
G_loss: 1.846

Iter: 24

#### The output is as follows:

<img src = 'https://lh5.googleusercontent.com/j1v0n4eRt-QlQOH4nN8_dO813e6aJWnpoeZYfFLSdS63KSD0NeDQwiWdWxhiR8VXQnGXgNkHTNOWiQ=w1902-h1443' width = "90%">